In [ ]:
#Import Libraries:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV

Part 1: Utilizing Logistic Regression

In [ ]:
diabetes_prediction_dataset = pd.read_csv('/content/diabetes_prediction_dataset.csv')

diabetes_prediction_dataset.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [ ]:
diabetes_prediction_dataset['diabetes'].value_counts()

0    91500
1     8500
Name: diabetes, dtype: int64

In [ ]:
diabetes_prediction_dataset['smoking_history'].value_counts()

No Info        35816
never          35095
former          9352
current         9286
not current     6447
ever            4004
Name: smoking_history, dtype: int64

In [ ]:
# Bonus: undersample the diabetes_prediction_dataset such that we have 8500 observations of diabetes = 0, and 8500 of diabetes = 1

df0 = diabetes_prediction_dataset[diabetes_prediction_dataset['diabetes']==0]
df1 = diabetes_prediction_dataset[diabetes_prediction_dataset['diabetes']==1]

df1_undersampled = df1.sample(n=8500, random_state=123)
df0_undersampled = df0.sample(n=8500, random_state=123)

diabetes_prediction_dataset_undersampled = pd.concat([df0_undersampled, df1_undersampled])

diabetes_prediction_dataset_undersampled = diabetes_prediction_dataset_undersampled.sample(frac=1, random_state=123).reset_index(drop=True)

diabetes_prediction_dataset = diabetes_prediction_dataset_undersampled

In [ ]:
diabetes_prediction_dataset

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Male,43.0,0,0,never,33.19,6.1,140,0
1,Female,80.0,1,0,No Info,27.32,7.5,220,1
2,Female,43.0,0,0,No Info,40.75,6.1,130,0
3,Female,55.0,0,0,never,42.64,6.0,155,1
4,Male,32.0,0,0,ever,27.32,4.5,130,0
...,...,...,...,...,...,...,...,...,...
16995,Male,14.0,0,0,No Info,21.46,6.0,90,0
16996,Female,80.0,1,0,never,23.04,8.8,130,1
16997,Male,49.0,0,0,former,32.11,4.8,159,0
16998,Male,52.0,0,0,current,28.88,8.2,160,1


In [ ]:
# Convert relevant variables to dummies.
diabetes_prediction_dataset = pd.get_dummies(diabetes_prediction_dataset, columns=['smoking_history', 'gender'], drop_first=True)

diabetes_prediction_dataset.head(20)

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current,gender_Male,gender_Other
0,43.0,0,0,33.19,6.1,140,0,0,0,0,1,0,1,0
1,80.0,1,0,27.32,7.5,220,1,0,0,0,0,0,0,0
2,43.0,0,0,40.75,6.1,130,0,0,0,0,0,0,0,0
3,55.0,0,0,42.64,6.0,155,1,0,0,0,1,0,0,0
4,32.0,0,0,27.32,4.5,130,0,0,1,0,0,0,1,0
5,12.0,0,0,19.11,5.8,100,0,0,0,0,0,0,1,0
6,65.0,0,1,28.36,7.5,220,1,0,0,1,0,0,0,0
7,75.0,1,0,25.37,6.2,155,1,0,0,0,0,0,0,0
8,52.0,0,0,43.21,7.0,160,1,0,0,0,0,0,0,0
9,68.0,0,0,26.74,4.0,100,0,0,0,0,1,0,0,0


In [ ]:
# Split training and testing sets
# Create the features DataFrame, X
X = diabetes_prediction_dataset.copy()
X = X.drop(columns='diabetes')

# Create the target DataFrame, y
y = diabetes_prediction_dataset['diabetes']

# Use train_test_split to separate the data
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 42)

In [ ]:
#fit logistic regression

logistic_regression_model = LogisticRegression(max_iter = 1000,random_state=1)
logistic_regression_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=1)

In [ ]:
#score logistic regression

print(f"Training Data Score: {logistic_regression_model.score(X_train, y_train)}")
print(f"Testing Data Score: {logistic_regression_model.score(X_test, y_test)}")

Training Data Score: 0.887686274509804
Testing Data Score: 0.8844705882352941


In [ ]:
# Generate predictions from the model we just fit
predictions = logistic_regression_model.predict(X_train)

# Convert those predictions (and actual values) to a DataFrame
results_df_train = pd.DataFrame({"Prediction": predictions, "Actual": y_train})
results_df_train

,Prediction,Actual
15234,1,1
10232,0,0
13469,0,0
16217,1,1
5128,0,0
...,...,...
13782,1,1
11022,1,1
16779,1,1
15012,0,0


In [ ]:
# Apply the fitted model to the `test` dataset
testing_predictions = logistic_regression_model.predict(X_test)

# Save both the test predictions and actual test values to a DataFrame
results_df_test = pd.DataFrame({
    "Testing Data Predictions": testing_predictions,
    "Testing Data Actual Targets": y_test})

results_df_test

,Testing Data Predictions,Testing Data Actual Targets
5191,0,0
13041,1,1
2098,0,0
10282,0,0
10217,1,1
...,...,...
15994,0,0
9871,0,0
2576,0,0
15107,0,0


In [ ]:
# Calculate the model's accuracy on the test dataset
accuracy_score(y_test, testing_predictions)

0.8844705882352941

In [ ]:
# generate a summary of the logistic regression model we made

print(logistic_regression_model.intercept_)

print(logistic_regression_model.coef_)

intercept = logistic_regression_model.intercept_

coefs = logistic_regression_model.coef_

[-24.53617263]
[[ 0.05181298  0.84486081  0.72148641  0.09567376  2.11906653  0.03296938
   0.83726142  0.6692182   0.71075976  0.63058089  0.63837717  0.29630892
  -0.0183521 ]]


In [ ]:
# formula for the model, *IF* it were a linear model:
print(intercept[0], "+")
for i in range(len(X_train.columns)):
  if i+1 < len(X_train.columns):
    print(f"{coefs[0][i]} * {X_train.columns[i]} + ")
  else:
    print(f"{coefs[0][i]} * {X_train.columns[i]}")

-24.53617263225597 +
0.051812980353859346 * age + 
0.844860812543107 * hypertension + 
0.7214864139177596 * heart_disease + 
0.09567376357816561 * bmi + 
2.1190665290598227 * HbA1c_level + 
0.03296938410024442 * blood_glucose_level + 
0.8372614226998919 * smoking_history_current + 
0.6692181960363439 * smoking_history_ever + 
0.7107597579286823 * smoking_history_former + 
0.630580894098804 * smoking_history_never + 
0.6383771670878637 * smoking_history_not current + 
0.2963089178855829 * gender_Male + 
-0.018352095050865457 * gender_Other


In [ ]:
# prompt: do a vif test of all the variables in X_train

vif = pd.DataFrame()
vif['features'] = X_train.columns
vif['vif_value'] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]

vif

,features,vif_value
0,age,8.766735
1,hypertension,1.248914
2,heart_disease,1.190533
3,bmi,14.916372
4,HbA1c_level,18.867755
5,blood_glucose_level,10.740396
6,smoking_history_current,1.389211
7,smoking_history_ever,1.191666
8,smoking_history_former,1.664112
9,smoking_history_never,2.509464


In [ ]:
# prompt: do a correlation plot of the X_train variables

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 10))
sns.heatmap(X_train.corr(), annot=True)
plt.show()


In [ ]:
# prompt: do a significance test of the variables in the logistic regression with max iterations set to 1000

import statsmodels.api as sm

# Add a constant term to the predictor matrix
X_train_sm = sm.add_constant(X_train)

# Fit the logistic regression model using statsmodels
logistic_regression_model_sm = sm.Logit(y_train, X_train_sm).fit(maxiter=2000,random_state=1)

# Print the summary of the logistic regression model
print(logistic_regression_model_sm.summary())

# Check the significance of the variables
for i, p_value in enumerate(logistic_regression_model_sm.pvalues[1:]):
    if p_value < 0.05:
        print(f"Variable '{X_train.columns[i]}' is significant (p-value = {p_value})")
    else:
        print(f"Variable '{X_train.columns[i]}' is not significant (p-value = {p_value})")


/usr/local/lib/python3.10/dist-packages/statsmodels/base/optimizer.py:18: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method newton is: tol, ridge_factor. The list of unsupported keyword arguments passed include: random_state. After release 0.14, this will raise.
  warnings.warn(


         Current function value: 0.242336
         Iterations: 2000
                           Logit Regression Results                           
Dep. Variable:               diabetes   No. Observations:                12750
Model:                          Logit   Df Residuals:                    12736
Method:                           MLE   Df Model:                           13
Date:                Thu, 29 Feb 2024   Pseudo R-squ.:                  0.6504
Time:                        02:05:31   Log-Likelihood:                -3089.8
converged:                      False   LL-Null:                       -8837.6
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                         -24.6545      0.518    -47.559      0.000     -25.671     -23.638
age         

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [ ]:
X_train["gender_Other"].value_counts()

0    12749
1        1
Name: gender_Other, dtype: int64

In [ ]:
# Input Euler's number and interpret logistic regression

e = 2.718281828459045235360

exponentiated_coefs = e*coefs

exponentiated_coefs

NameError: name 'coefs' is not defined

In [ ]:


def diabetes_risk_score(age, hypertension, heart_disease, bmi, HbA1c_level, blood_glucose_level, smoking_history_current, smoking_history_ever, smoking_history_former, smoking_history_never, smoking_history_not_current, gender_Male, gender_Other):

  score = intercept[0] + coefs[0][0]*age + coefs[0][1]*hypertension + coefs[0][2]*heart_disease + coefs[0][3]*bmi + coefs[0][4]*HbA1c_level + coefs[0][5]*blood_glucose_level + coefs[0][6] * smoking_history_current + coefs[0][7]*smoking_history_ever + coefs[0][8]*smoking_history_former + coefs[0][9] * smoking_history_never + coefs[0][10] * smoking_history_not_current + coefs[0][11]*gender_Male + coefs[0][12]*gender_Other
  logit_odds = e**(score) / (e**(score)+1)

  return logit_odds


In [ ]:
#let's get some logit odds for toy values and manually calculate to help us understand:

#the following were observations were diabetes was observed
print('diabetes observations oddsrange:')
number_393 = diabetes_risk_score(age = 29,hypertension = 0,heart_disease = 0,bmi = 25.41,HbA1c_level = 6.1,blood_glucose_level=130,smoking_history_current = 1, smoking_history_ever=0,smoking_history_former=0,smoking_history_never=0,smoking_history_not_current=0,gender_Male=1,gender_Other=0)
print(number_393)
number_43 = diabetes_risk_score(age = 44,hypertension = 0,heart_disease = 0,bmi = 19.31,HbA1c_level = 6.5,blood_glucose_level=200,smoking_history_current = 0, smoking_history_ever=0,smoking_history_former=0,smoking_history_never=1,smoking_history_not_current=0,gender_Male=0,gender_Other=0)
print(number_43)
number_65534 = number_43 = diabetes_risk_score(age = 80,hypertension = 0,heart_disease = 0,bmi = 29.81,HbA1c_level = 5.7,blood_glucose_level=126,smoking_history_current = 0, smoking_history_ever=0,smoking_history_former=1,smoking_history_never=1,smoking_history_not_current=0,gender_Male=0,gender_Other=0)
print(number_65534)
print()

#let's compare where diabetes was not observed, given the raw data
print('no diabetes oddsrange:')
number_31061 = diabetes_risk_score(age = 20,hypertension = 0,heart_disease = 0,bmi = 19.6,HbA1c_level = 6.1,blood_glucose_level=100,smoking_history_current = 0, smoking_history_ever=0,smoking_history_former=0,smoking_history_never=0,smoking_history_not_current=0,gender_Male=0,gender_Other=0)
print(number_31061)
number_98396 = diabetes_risk_score(age = 45,hypertension = 0,heart_disease = 0,bmi = 23.11,HbA1c_level = 4.8,blood_glucose_level=126,smoking_history_current = 1, smoking_history_ever=0,smoking_history_former=0,smoking_history_never=0,smoking_history_not_current=0,gender_Male=0,gender_Other=0)
print(number_98396)
number_4700 = diabetes_risk_score(age = 61,hypertension = 0,heart_disease = 0,bmi = 27.05,HbA1c_level = 5,blood_glucose_level=160,smoking_history_current = 0, smoking_history_ever=0,smoking_history_former=0,smoking_history_never=1,smoking_history_not_current=0,gender_Male=0,gender_Other=0)
print(number_4700)
print()

#restrict to observations where diabetes was observed and predicted
print('observations that are true positives for diabetes')
number_3070 = diabetes_risk_score(age = 71,hypertension = 1,heart_disease = 1,bmi = 37.1,HbA1c_level = 6.1,blood_glucose_level=159,smoking_history_current = 0, smoking_history_ever=0,smoking_history_former=0,smoking_history_never=0,smoking_history_not_current=0,gender_Male=1,gender_Other=0)
print(number_3070)
number_26673 = diabetes_risk_score(age = 77,hypertension = 0,heart_disease = 0,bmi = 26.23,HbA1c_level = 8.8,blood_glucose_level=300,smoking_history_current = 0, smoking_history_ever=0,smoking_history_former=1,smoking_history_never=0,smoking_history_not_current=0,gender_Male=0,gender_Other=0)
print(number_26673)
number_89370 = diabetes_risk_score(age = 58,hypertension = 0,heart_disease = 1,bmi = 36.81,HbA1c_level = 9,blood_glucose_level=145,smoking_history_current = 0, smoking_history_ever=0,smoking_history_former=0,smoking_history_never=0,smoking_history_not_current=1,gender_Male=0,gender_Other=0)
print(number_89370)

diabetes observations oddsrange:
0.09477912684576165
0.6432387178485202
0.508764840207036

no diabetes oddsrange:
0.004489729337617444
0.007918176237698664
0.09223963326196119

observations that are true positives for diabetes
0.9383783320229956
0.9999864932854782
0.9992560217502745


In [ ]:
# use results_df_test to make a confusion matrix

# Create a confusion matrix
confusion_matrix = confusion_matrix(results_df_test["Testing Data Actual Targets"], results_df_test["Testing Data Predictions"])

# Print the confusion matrix
print(confusion_matrix)

# Convert the confusion matrix to a DataFrame
confusion_matrix_df = pd.DataFrame(confusion_matrix, columns=["Predicted No Diabetes", "Predicted Diabetes"], index=["Actual No Diabetes", "Actual Diabetes"])

# Print the confusion matrix DataFrame
print(confusion_matrix_df)


[[1893  249]
 [ 242 1866]]
                    Predicted No Diabetes  Predicted Diabetes
Actual No Diabetes                   1893                 249
Actual Diabetes                       242                1866


Part 2: Trying different Model Techniques

In [ ]:
# Read in the Dataset
diabetes_prediction_dataset = pd.read_csv('/content/diabetes_prediction_dataset.csv')

diabetes_prediction_dataset.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [ ]:
diabetes_prediction_dataset = pd.get_dummies(diabetes_prediction_dataset, columns=['smoking_history', 'gender'], drop_first=False)

diabetes_prediction_dataset.head()

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes,smoking_history_No Info,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current,gender_Female,gender_Male,gender_Other
0,80.0,0,1,25.19,6.6,140,0,0,0,0,0,1,0,1,0,0
1,54.0,0,0,27.32,6.6,80,0,1,0,0,0,0,0,1,0,0
2,28.0,0,0,27.32,5.7,158,0,0,0,0,0,1,0,0,1,0
3,36.0,0,0,23.45,5.0,155,0,0,1,0,0,0,0,1,0,0
4,76.0,1,1,20.14,4.8,155,0,0,1,0,0,0,0,0,1,0


In [ ]:
diabetes_prediction_dataset['diabetes'].value_counts()

0    91500
1     8500
Name: diabetes, dtype: int64

In [ ]:
# prompt: split the diabetes_prediction_dataset into train and test data with random_state = 42 and use a random forrest classifier on it to predict diabetis based on the other variables. Then analyze the model fit and make a confusion matrix to score it

# Split the data into training and test sets
X = diabetes_prediction_dataset.drop('diabetes', axis=1)
y = diabetes_prediction_dataset['diabetes']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Train a random forest classifier

random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train, y_train)

# Analyze the model fit

y_pred_rf = random_forest_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Accuracy: {accuracy_rf}")

print(classification_report(y_test, y_pred_rf))

# Create a confusion matrix
confusion_matrix_rf = confusion_matrix(y_test, y_pred_rf)

# Print the confusion matrix
print(confusion_matrix_rf)

# Convert the confusion matrix to a DataFrame
confusion_matrix_df_rf = pd.DataFrame(confusion_matrix_rf, columns=["Predicted No Diabetes", "Predicted Diabetes"], index=["Actual No Diabetes", "Actual Diabetes"])

# Print the confusion matrix DataFrame
print(confusion_matrix_df_rf)

Accuracy: 0.96968
              precision    recall  f1-score   support

           0       0.97      1.00      0.98     22850
           1       0.95      0.68      0.79      2150

    accuracy                           0.97     25000
   macro avg       0.96      0.84      0.89     25000
weighted avg       0.97      0.97      0.97     25000

[[22777    73]
 [  685  1465]]
                    Predicted No Diabetes  Predicted Diabetes
Actual No Diabetes                  22777                  73
Actual Diabetes                       685                1465


In [ ]:
# prompt: do the same analysis again using a support vector machine

# Import the necessary libraries
from sklearn.svm import SVC

# Create a support vector machine classifier
svm_model = SVC(random_state=42)

# Train the model
svm_model.fit(X_train, y_train)

# Analyze the model fit

# Make predictions on the test set
y_pred_svm = svm_model.predict(X_test)

# Calculate the accuracy of the model
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"Accuracy: {accuracy_svm}")

# Print the classification report
print(classification_report(y_test, y_pred_svm))

# Create a confusion matrix
confusion_matrix_svm = confusion_matrix(y_test, y_pred_svm)

# Print the confusion matrix
print(confusion_matrix_svm)

# Convert the confusion matrix to a DataFrame
confusion_matrix_df_svm = pd.DataFrame(confusion_matrix_svm, columns=["Predicted No Diabetes", "Predicted Diabetes"], index=["Actual No Diabetes", "Actual Diabetes"])

# Print the confusion matrix DataFrame
print(confusion_matrix_df_svm)


Accuracy: 0.94532
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     22850
           1       1.00      0.36      0.53      2150

    accuracy                           0.95     25000
   macro avg       0.97      0.68      0.75     25000
weighted avg       0.95      0.95      0.93     25000

[[22850     0]
 [ 1367   783]]
                    Predicted No Diabetes  Predicted Diabetes
Actual No Diabetes                  22850                   0
Actual Diabetes                      1367                 783


In [ ]:
# prompt: do the same analysis again, this time using a gradient boosted model

# Import the necessary libraries
from sklearn.ensemble import GradientBoostingClassifier

# Create a gradient boosted classifier
gbm_model = GradientBoostingClassifier(random_state=42)

# Train the model
gbm_model.fit(X_train, y_train)

# Analyze the model fit

# Make predictions on the test set
y_pred_gbm = gbm_model.predict(X_test)

# Calculate the accuracy of the model
accuracy_gbm = accuracy_score(y_test, y_pred_gbm)
print(f"Accuracy: {accuracy_gbm}")

# Print the classification report
print(classification_report(y_test, y_pred_gbm))

# Create a confusion matrix
confusion_matrix_gbm = confusion_matrix(y_test, y_pred_gbm)

# Print the confusion matrix
print(confusion_matrix_gbm)

# Convert the confusion matrix to a DataFrame
confusion_matrix_df_gbm = pd.DataFrame(confusion_matrix_gbm, columns=["Predicted No Diabetes", "Predicted Diabetes"], index=["Actual No Diabetes", "Actual Diabetes"])

# Print the confusion matrix DataFrame
print(confusion_matrix_df_gbm)


Accuracy: 0.97124
              precision    recall  f1-score   support

           0       0.97      1.00      0.98     22850
           1       0.99      0.68      0.80      2150

    accuracy                           0.97     25000
   macro avg       0.98      0.84      0.89     25000
weighted avg       0.97      0.97      0.97     25000

[[22829    21]
 [  698  1452]]
                    Predicted No Diabetes  Predicted Diabetes
Actual No Diabetes                  22829                  21
Actual Diabetes                       698                1452


In [ ]:
# prompt: Do the same analysis again, this time using k nearest neighbor

# Import the necessary libraries

from sklearn.neighbors import KNeighborsClassifier

# Create a k-nearest neighbors classifier
knn_model = KNeighborsClassifier(n_neighbors=3)

# Train the model
knn_model.fit(X_train, y_train)

# Analyze the model fit

# Make predictions on the test set
y_pred_knn = knn_model.predict(X_test)

# Calculate the accuracy of the model
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"Accuracy: {accuracy_knn}")

# Print the classification report
print(classification_report(y_test, y_pred_knn))

# Create a confusion matrix
confusion_matrix_knn = confusion_matrix(y_test, y_pred_knn)

# Print the confusion matrix
print(confusion_matrix_knn)

# Convert the confusion matrix to a DataFrame
confusion_matrix_df_knn = pd.DataFrame(confusion_matrix_knn, columns=["Predicted No Diabetes", "Predicted Diabetes"], index=["Actual No Diabetes", "Actual Diabetes"])

# Print the confusion matrix DataFrame
print(confusion_matrix_df_knn)


Accuracy: 0.95008
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     22850
           1       0.82      0.54      0.65      2150

    accuracy                           0.95     25000
   macro avg       0.89      0.76      0.81     25000
weighted avg       0.95      0.95      0.95     25000

[[22601   249]
 [  999  1151]]
                    Predicted No Diabetes  Predicted Diabetes
Actual No Diabetes                  22601                 249
Actual Diabetes                       999                1151


In [ ]:
# prompt: do the same analysis again, this time using a neural network and import the necessary libraries

# Import the necessary libraries
from sklearn.neural_network import MLPClassifier

# Create a neural network classifier
nn_model = MLPClassifier(random_state=42)

# Train the model
nn_model.fit(X_train, y_train)

# Analyze the model fit

# Make predictions on the test set
y_pred_nn = nn_model.predict(X_test)

# Calculate the accuracy of the model
accuracy_nn = accuracy_score(y_test, y_pred_nn)
print(f"Accuracy: {accuracy_nn}")

# Print the classification report
print(classification_report(y_test, y_pred_nn))

# Create a confusion matrix
confusion_matrix_nn = confusion_matrix(y_test, y_pred_nn)

# Print the confusion matrix
print(confusion_matrix_nn)

# Convert the confusion matrix to a DataFrame
confusion_matrix_df_nn = pd.DataFrame(confusion_matrix_nn, columns=["Predicted No Diabetes", "Predicted Diabetes"], index=["Actual No Diabetes", "Actual Diabetes"])

# Print the confusion matrix DataFrame
print(confusion_matrix_df_nn)


Accuracy: 0.96384
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     22850
           1       0.87      0.68      0.76      2150

    accuracy                           0.96     25000
   macro avg       0.92      0.84      0.87     25000
weighted avg       0.96      0.96      0.96     25000

[[22633   217]
 [  687  1463]]
                    Predicted No Diabetes  Predicted Diabetes
Actual No Diabetes                  22633                 217
Actual Diabetes                       687                1463


Part 3 : oversampling / undersampling the data

```
# This is formatted as code
```



In [ ]:
# prompt: undersample the data and run the random forest classifier again

df0 = diabetes_prediction_dataset[diabetes_prediction_dataset['diabetes']==0]
df1 = diabetes_prediction_dataset[diabetes_prediction_dataset['diabetes']==1]

df1_oversampled = df1.sample(n=8500, random_state=123, replace=False)
df0_oversampled = df0.sample(n=8500, random_state=123, replace=False)

diabetes_prediction_dataset_oversampled = pd.concat([df0_oversampled, df1_oversampled])

diabetes_prediction_dataset_oversampled = diabetes_prediction_dataset_oversampled.sample(frac=1, random_state=123).reset_index(drop=True)

diabetes_prediction_dataset = diabetes_prediction_dataset_oversampled
diabetes_prediction_dataset

# Split the data into training and test sets
X = diabetes_prediction_dataset.drop('diabetes', axis=1)
y = diabetes_prediction_dataset['diabetes']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Train a random forest classifier

random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train, y_train)

# Analyze the model fit

y_pred_rf = random_forest_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Accuracy: {accuracy_rf}")

print(classification_report(y_test, y_pred_rf))

# Import the confusion matrix function
from sklearn.metrics import confusion_matrix

# Create a confusion matrix
confusion_matrix_rf = confusion_matrix(y_test, y_pred_rf)

# Print the confusion matrix
print(confusion_matrix_rf)

# Convert the confusion matrix to a DataFrame
confusion_matrix_df_rf = pd.DataFrame(confusion_matrix_rf, columns=["Predicted No Diabetes", "Predicted Diabetes"], index=["Actual No Diabetes", "Actual Diabetes"])

# Print the confusion matrix
print(confusion_matrix_rf)

# Convert the confusion matrix to a DataFrame
confusion_matrix_df_rf = pd.DataFrame(confusion_matrix_rf, columns=["Predicted No Diabetes", "Predicted Diabetes"], index=["Actual No Diabetes", "Actual Diabetes"])

# Print the confusion matrix DataFrame
print(confusion_matrix_df_rf)

Accuracy: 0.9016470588235294
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      2085
           1       0.91      0.90      0.90      2165

    accuracy                           0.90      4250
   macro avg       0.90      0.90      0.90      4250
weighted avg       0.90      0.90      0.90      4250

[[1884  201]
 [ 217 1948]]
[[1884  201]
 [ 217 1948]]
                    Predicted No Diabetes  Predicted Diabetes
Actual No Diabetes                   1884                 201
Actual Diabetes                       217                1948


In [ ]:
diabetes_prediction_dataset = pd.read_csv('/content/diabetes_prediction_dataset.csv')
diabetes_prediction_dataset = pd.get_dummies(diabetes_prediction_dataset, columns=['smoking_history', 'gender'], drop_first=False)

In [ ]:
#Now oversample the data

df0 = diabetes_prediction_dataset[diabetes_prediction_dataset['diabetes']==0]
df1 = diabetes_prediction_dataset[diabetes_prediction_dataset['diabetes']==1]

df1_oversampled = df1.sample(n=91500, random_state=123, replace=True)
df0_oversampled = df0.sample(n=91500, random_state=123, replace=False)

diabetes_prediction_dataset_oversampled = pd.concat([df0_oversampled, df1_oversampled])

diabetes_prediction_dataset_oversampled = diabetes_prediction_dataset_oversampled.sample(frac=1, random_state=123).reset_index(drop=True)

diabetes_prediction_dataset = diabetes_prediction_dataset_oversampled
diabetes_prediction_dataset

# Split the data into training and test sets
X = diabetes_prediction_dataset.drop('diabetes', axis=1)
y = diabetes_prediction_dataset['diabetes']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Train a random forest classifier

random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train, y_train)

# Analyze the model fit

y_pred_rf = random_forest_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Accuracy: {accuracy_rf}")

print(classification_report(y_test, y_pred_rf))

# Import the confusion matrix function
from sklearn.metrics import confusion_matrix

# Create a confusion matrix
confusion_matrix_rf = confusion_matrix(y_test, y_pred_rf)

# Print the confusion matrix
print(confusion_matrix_rf)

# Convert the confusion matrix to a DataFrame
confusion_matrix_df_rf = pd.DataFrame(confusion_matrix_rf, columns=["Predicted No Diabetes", "Predicted Diabetes"], index=["Actual No Diabetes", "Actual Diabetes"])

# Print the confusion matrix
print(confusion_matrix_rf)

# Convert the confusion matrix to a DataFrame
confusion_matrix_df_rf = pd.DataFrame(confusion_matrix_rf, columns=["Predicted No Diabetes", "Predicted Diabetes"], index=["Actual No Diabetes", "Actual Diabetes"])

# Print the confusion matrix DataFrame
print(confusion_matrix_df_rf)

Accuracy: 0.9900327868852459
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     22784
           1       0.98      1.00      0.99     22966

    accuracy                           0.99     45750
   macro avg       0.99      0.99      0.99     45750
weighted avg       0.99      0.99      0.99     45750

[[22328   456]
 [    0 22966]]
[[22328   456]
 [    0 22966]]
                    Predicted No Diabetes  Predicted Diabetes
Actual No Diabetes                  22328                 456
Actual Diabetes                         0               22966


Part 4: Go back to regular dataset and use the random forrest classifier and tweak hyperparameters as I am suspicious of hte .99 model

In [ ]:
diabetes_prediction_dataset = pd.read_csv('/content/diabetes_prediction_dataset.csv')
diabetes_prediction_dataset = pd.get_dummies(diabetes_prediction_dataset, columns=['smoking_history', 'gender'], drop_first=False)

In [ ]:
# prompt: without using any oversampling, create a random forest model that utilizes hyperparameter tuning to find the best fit


# Create a random forest classifier
random_forest_model = RandomForestClassifier()

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 150, 200, 250, 300],
    'max_depth': [5, 7 ,9 , 11],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform grid search cross-validation
grid_search = GridSearchCV(random_forest_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Print the classification report
print(classification_report(y_test, y_pred))

# Create a confusion matrix
confusion_matrix = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print(confusion_matrix)

# Convert the confusion matrix to a DataFrame
confusion_matrix_df = pd.DataFrame(confusion_matrix, columns=["Predicted No Diabetes", "Predicted Diabetes"], index=["Actual No Diabetes", "Actual Diabetes"])

# Print the confusion matrix DataFrame
print(confusion_matrix_df)


Accuracy: 0.9139672131147542
              precision    recall  f1-score   support

           0       0.92      0.90      0.91     22784
           1       0.90      0.93      0.92     22966

    accuracy                           0.91     45750
   macro avg       0.91      0.91      0.91     45750
weighted avg       0.91      0.91      0.91     45750

[[20513  2271]
 [ 1665 21301]]
                    Predicted No Diabetes  Predicted Diabetes
Actual No Diabetes                  20513                2271
Actual Diabetes                      1665               21301


In [ ]:
# prompt: now undersample the data to 8500 in each

df0 = diabetes_prediction_dataset[diabetes_prediction_dataset['diabetes']==0]
df1 = diabetes_prediction_dataset[diabetes_prediction_dataset['diabetes']==1]

df1_undersampled = df1.sample(n=8500, random_state=123, replace=False)
df0_undersampled = df0.sample(n=8500, random_state=123, replace=False)

diabetes_prediction_dataset_undersampled = pd.concat([df0_undersampled, df1_undersampled])

diabetes_prediction_dataset_undersampled = diabetes_prediction_dataset_undersampled.sample(frac=1, random_state=123).reset_index(drop=True)

diabetes_prediction_dataset = diabetes_prediction_dataset_undersampled
diabetes_prediction_dataset


,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes,smoking_history_No Info,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current,gender_Female,gender_Male,gender_Other
0,43.0,0,0,33.19,6.1,140,0,0,0,0,0,1,0,0,1,0
1,80.0,1,0,27.32,7.5,220,1,1,0,0,0,0,0,1,0,0
2,43.0,0,0,40.75,6.1,130,0,1,0,0,0,0,0,1,0,0
3,55.0,0,0,42.64,6.0,155,1,0,0,0,0,1,0,1,0,0
4,32.0,0,0,27.32,4.5,130,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,14.0,0,0,21.46,6.0,90,0,1,0,0,0,0,0,0,1,0
16996,80.0,1,0,23.04,8.8,130,1,0,0,0,0,1,0,1,0,0
16997,49.0,0,0,32.11,4.8,159,0,0,0,0,1,0,0,0,1,0
16998,52.0,0,0,28.88,8.2,160,1,0,1,0,0,0,0,0,1,0


In [ ]:
# prompt: split the data into training and testing and hyperparameter tune and fit a random forest model with the help of cross validation

# Split the data into training and testing sets
X = diabetes_prediction_dataset.drop('diabetes', axis=1)
y = diabetes_prediction_dataset['diabetes']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Create a random forest classifier
random_forest_model = RandomForestClassifier(random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 150, 200, 250, 300],
    'max_depth': [5, 7 ,9 , 11],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform grid search cross-validation
grid_search = GridSearchCV(random_forest_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Print the classification report
print(classification_report(y_test, y_pred))

# Create a confusion matrix
confusion_matrix = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print(confusion_matrix)

# Convert the confusion matrix to a DataFrame
confusion_matrix_df = pd.DataFrame(confusion_matrix, columns=["Predicted No Diabetes", "Predicted Diabetes"], index=["Actual No Diabetes", "Actual Diabetes"])

# Print the confusion matrix DataFrame
print(confusion_matrix_df)


Accuracy: 0.9101176470588235
              precision    recall  f1-score   support

           0       0.91      0.90      0.91      2085
           1       0.91      0.92      0.91      2165

    accuracy                           0.91      4250
   macro avg       0.91      0.91      0.91      4250
weighted avg       0.91      0.91      0.91      4250

[[1884  201]
 [ 181 1984]]
                    Predicted No Diabetes  Predicted Diabetes
Actual No Diabetes                   1884                 201
Actual Diabetes                       181                1984


In [ ]:
diabetes_prediction_dataset = pd.read_csv('/content/diabetes_prediction_dataset.csv')
diabetes_prediction_dataset = pd.get_dummies(diabetes_prediction_dataset, columns=['smoking_history', 'gender'], drop_first=False)

# prompt: now undersample the data to 8500 in each

df0 = diabetes_prediction_dataset[diabetes_prediction_dataset['diabetes']==0]
df1 = diabetes_prediction_dataset[diabetes_prediction_dataset['diabetes']==1]

df1_undersampled = df1.sample(n=91500, random_state=123, replace=True)
df0_undersampled = df0.sample(n=91500, random_state=123, replace=False)

diabetes_prediction_dataset_undersampled = pd.concat([df0_undersampled, df1_undersampled])

diabetes_prediction_dataset_undersampled = diabetes_prediction_dataset_undersampled.sample(frac=1, random_state=123).reset_index(drop=True)

diabetes_prediction_dataset = diabetes_prediction_dataset_undersampled
diabetes_prediction_dataset

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes,smoking_history_No Info,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current,gender_Female,gender_Male,gender_Other
0,3.0,0,0,21.87,4.8,145,0,1,0,0,0,0,0,0,1,0
1,44.0,0,0,27.32,6.8,140,1,0,0,0,0,1,0,1,0,0
2,77.0,0,1,39.35,6.6,260,1,0,0,0,0,1,0,1,0,0
3,56.0,0,0,27.32,6.6,80,0,1,0,0,0,0,0,0,1,0
4,45.0,0,0,37.73,8.2,130,1,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182995,75.0,1,0,21.45,8.8,159,1,0,0,0,0,1,0,1,0,0
182996,69.0,0,0,36.59,6.1,140,1,0,0,0,1,0,0,0,1,0
182997,80.0,0,0,24.79,6.2,155,0,0,0,0,0,1,0,1,0,0
182998,24.0,0,0,23.73,6.0,126,0,0,0,1,0,0,0,0,1,0


In [ ]:
# prompt: split the data into training and testing and hyperparameter tune and fit a random forest model with the help of cross validation

# Split the data into training and testing sets
X = diabetes_prediction_dataset.drop('diabetes', axis=1)
y = diabetes_prediction_dataset['diabetes']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Create a random forest classifier
random_forest_model = RandomForestClassifier(random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 150, 200, 250],
    'max_depth': [5, 7 ,9 , 11],
    'min_samples_split': [3, 5, 7],
    'min_samples_leaf': [2, 4]
}

# Perform grid search cross-validation
grid_search = GridSearchCV(random_forest_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Print the classification report
print(classification_report(y_test, y_pred))

# Create a confusion matrix
confusion_matrix = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print(confusion_matrix)

# Convert the confusion matrix to a DataFrame
confusion_matrix_df = pd.DataFrame(confusion_matrix, columns=["Predicted No Diabetes", "Predicted Diabetes"], index=["Actual No Diabetes", "Actual Diabetes"])

# Print the confusion matrix DataFrame
print(confusion_matrix_df)

Accuracy: 0.9139672131147542
              precision    recall  f1-score   support

           0       0.92      0.91      0.91     22784
           1       0.91      0.92      0.91     22966

    accuracy                           0.91     45750
   macro avg       0.91      0.91      0.91     45750
weighted avg       0.91      0.91      0.91     45750

[[20632  2152]
 [ 1784 21182]]
                    Predicted No Diabetes  Predicted Diabetes
Actual No Diabetes                  20632                2152
Actual Diabetes                      1784               21182


In [ ]:
# prompt: create the same thing again, only searching a much wider hyperparameter grid, but using randomized grid search instead of grid search

diabetes_prediction_dataset = pd.read_csv('/content/diabetes_prediction_dataset.csv')
diabetes_prediction_dataset = pd.get_dummies(diabetes_prediction_dataset, columns=['smoking_history', 'gender'], drop_first=False)


df0 = diabetes_prediction_dataset[diabetes_prediction_dataset['diabetes']==0]
df1 = diabetes_prediction_dataset[diabetes_prediction_dataset['diabetes']==1]

df1_undersampled = df1.sample(n=91500, random_state=123, replace=True)
df0_undersampled = df0.sample(n=91500, random_state=123, replace=False)

diabetes_prediction_dataset_undersampled = pd.concat([df0_undersampled, df1_undersampled])

diabetes_prediction_dataset_undersampled = diabetes_prediction_dataset_undersampled.sample(frac=1, random_state=123).reset_index(drop=True)

diabetes_prediction_dataset = diabetes_prediction_dataset_undersampled
diabetes_prediction_dataset

# Split the data into training and testing sets
X = diabetes_prediction_dataset.drop('diabetes', axis=1)
y = diabetes_prediction_dataset['diabetes']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Create a random forest classifier
random_forest_model = RandomForestClassifier(random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 150, 200, 250, 300, 350, 400, 450, 500],
    'max_depth': [5, 7, 9, 11, 13, 15, 17, 19, 21],
    'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'max_features': ['sqrt', 'log2']
}

# Perform randomized grid search cross-validation
random_search = RandomizedSearchCV(random_forest_model, param_grid, cv=5, n_iter=20)
random_search.fit(X_train, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Print the classification report
print(classification_report(y_test, y_pred))

# Create a confusion matrix
confusion_matrix = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print(confusion_matrix)

# Convert the confusion matrix to a DataFrame
confusion_matrix_df = pd.DataFrame(confusion_matrix, columns=["Predicted No Diabetes", "Predicted Diabetes"], index=["Actual No Diabetes", "Actual Diabetes"])

# Print the confusion matrix DataFrame
print(confusion_matrix_df)


Accuracy: 0.9834535519125683
              precision    recall  f1-score   support

           0       1.00      0.97      0.98     22784
           1       0.97      1.00      0.98     22966

    accuracy                           0.98     45750
   macro avg       0.98      0.98      0.98     45750
weighted avg       0.98      0.98      0.98     45750

[[22073   711]
 [   46 22920]]
                    Predicted No Diabetes  Predicted Diabetes
Actual No Diabetes                  22073                 711
Actual Diabetes                        46               22920
